In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import tensorflow as tf
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Input
from keras.layers import Dense
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report

In [2]:
# Load the dataset
df = pd.read_csv('Steel-Faults.nna')

In [5]:
# Open the file and read contents
with open('Steel-Faults.nna', 'r') as f:
    contents = f.readlines()

# Process each line
data = []
for line in contents:
    # Split each line by tabs and convert to a list of values
    values = line.strip().split('\t')
    data.append(values)

# Print processed data
# for row in data:
#    print(row)

In [6]:
# Convert to DataFrame
df = pd.DataFrame(data, dtype=float,)
df.columns = ["X_Minimum","X_Maximum","Y_Minimum","Y_Maximum","Pixels_Areas","X_Perimeter","Y_Perimeter","Sum_of_Luminosity","Minimum_of_Luminosity","Maximum_of_Luminosity","Length_of_Conveyer","TypeOfSteel_A300","TypeOfSteel_A400","Steel_Plate_Thickness","Edges_Index","Empty_Index","Square_Index","Outside_X_Index","Edges_X_Index","Edges_Y_Index","Outside_Global_Index","LogOfAreas","Log_X_Index","Log_Y_Index","Orientation_Index","Luminosity_Index","SigmoidOfAreas","Pastry","Z_Scratch","K_Scratch","Stains","Dirtiness","Bumps","Other_Faults"]

df.head()

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,...,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scratch,Stains,Dirtiness,Bumps,Other_Faults
0,42.0,50.0,270900.0,270944.0,267.0,17.0,44.0,24220.0,76.0,108.0,...,0.8182,-0.2913,0.5822,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,645.0,651.0,2538079.0,2538108.0,108.0,10.0,30.0,11397.0,84.0,123.0,...,0.7931,-0.1756,0.2984,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,829.0,835.0,1553913.0,1553931.0,71.0,8.0,19.0,7972.0,99.0,125.0,...,0.6667,-0.1228,0.2150,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,853.0,860.0,369370.0,369415.0,176.0,13.0,45.0,18996.0,99.0,126.0,...,0.8444,-0.1568,0.5212,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1289.0,1306.0,498078.0,498335.0,2409.0,60.0,260.0,246930.0,37.0,126.0,...,0.9338,-0.1992,1.0000,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# Drop categorical variables
X = df.drop(['TypeOfSteel_A300', 'TypeOfSteel_A400'], axis=1)

# the categorical variables are dropped as we are focusing on numeric attributes only.

# Split features and labels
y = df[['Pastry', 'Z_Scratch', 'K_Scratch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults']]
X = X.drop(['Pastry', 'Z_Scratch', 'K_Scratch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults'], axis=1)

# One-hot encode the labels (y) since they are multi-class.
# One-hot encode labels
y = pd.get_dummies(y, columns=y.columns)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [39]:
# Define the input shape
input_shape = (X_train.shape[1],)

# Create the model
model = Sequential()
model.add(Input(shape=input_shape))
model.add(Dense(64, activation='relu')) # Hidden layer
model.add(Dense(32, activation='relu')) # Hidden layer
model.add(Dense(y_train.shape[1], activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

Epoch 1/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2490 - loss: 18.6248 - val_accuracy: 0.5013 - val_loss: 18.9658
Epoch 2/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 976us/step - accuracy: 0.7162 - loss: 21.6508 - val_accuracy: 0.9100 - val_loss: 37.0176
Epoch 3/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 886us/step - accuracy: 0.8660 - loss: 45.7427 - val_accuracy: 0.7815 - val_loss: 87.5923
Epoch 4/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 906us/step - accuracy: 0.7379 - loss: 107.3940 - val_accuracy: 0.7147 - val_loss: 182.4337
Epoch 5/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 978us/step - accuracy: 0.7076 - loss: 213.5909 - val_accuracy: 0.7224 - val_loss: 317.5440
Epoch 6/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 911us/step - accuracy: 0.6982 - loss: 351.3470 - val_accuracy: 0.6838 - val_loss: 479.9661
Epoch 7/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6820 - loss: 530.9420 - val_accuracy: 0.6581 - val_loss: 624.3719
Epoch 8/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6674 - loss: 6

In [40]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {test_accuracy:.2f}")
print(f'Test loss: {test_loss:.4f}')

# Predict the labels for the test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test.values, axis=1)

# Generate the classification report
print(classification_report(y_test_classes, y_pred_classes, zero_division=0))


Test Accuracy: 0.50
Test loss: 18.9658
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
              precision    recall  f1-score   support

           0       0.88      0.54      0.67       360
           1       0.00      0.00      0.00        29
           2       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
          10       0.00      0.00      0.00         0
          12       0.00      0.00      0.00         0
          13       0.00      0.00      0.00         0

    accuracy                           0.50       389
   macro avg       0.09      0.05      0.07       389
weighted avg       0.82      0.50      0.62       389



### Summary of returns
Test Accuracy: 0.50 which means it correctly classified 50% of the instances.

Test Loss: 18.9658
The loss value is low, indicating that the  model's predictions match the actual labels at a decent rate.

Classification Report
The classification report provides precision, recall, and F1-score for each class, along with support (the number of instances in the test set for each class).

For class 0 (likely "Pastry" fault):
Precision: 0.88 (88% of instances predicted as class 0 were actually class 0)
Recall: 0.54 (54% of actual class 0 instances were correctly predicted)
F1-score: 0.67 (harmonic mean of precision and recall)
Support: 360 instances

For class 1 (likely "Z_Scratch" fault):
Precision, recall, and F1-score are all 0.00, indicating that the model failed to correctly predict any instances.
Support: 29 instances

For classes 2 and 4 (likely "K_Scratch" and "Stains" faults):
Precision, recall, and F1-score are all 0.00, indicating that the model failed to correctly predict any instances of these classes.
Support: 0 instances these classes may not have been present in the test set

The model performs well for class 0 but fails to identify class 1 correctly and has no samples for class 2 and class 4 in the test set. 

For classes  6, 7, 8, 10, 12, 13:
Precision, recall, and F1-score are all 0.00, indicating that the model failed to correctly predict any instances of these classes.
Support: 0 instances these classes may not have been present in the test set
There are no samples of these classes in the test set, so these metrics are not applicable

Macro Average:

Precision: 0.09, Recall: 0.05 ,F1-Score: 0.07
These averages are low, reflecting poor performance for classes other than class 0.

Weighted Average:

Precision: 0.82, Recall: 0.50, F1-Score: 0.62
These averages are weighted by support (the number of true instances for each class). They are higher due to the large number of class 0 samples dominating the metrics.

In summary, the model performed well in predicting the "Pastry" fault (class 0) but struggled with the other fault types, particularly "Z_Scratch" (class 1). The lower test loss indicates better prediction quality compared to the previous model, but the accuracy is still moderate.